### 과제 : 프랙털 시각화

In [3]:
import numpy as np  
import matplotlib.pyplot as plt  
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 13
plt.ion()

간단한 복소수 함수 살펴보기

In [5]:
def f(z):
    return z**2 - 1
f(0)

-1

In [6]:
list = [4, 1-0.2j, 1.6]
array = np.array(list)
f(array)

array([15.  +0.j , -0.04-0.4j,  1.56+0.j ])


더 많은 복소수 집합에 대한 함수의 동작 확인하기

In [7]:
x_values = np.linspace(-10, 10, 20)
y_values = np.linspace(-10, 10, 20)

x, y = np.meshgrid(x_values, y_values)

In [8]:
mesh = x + y * 1j
function_result = f(mesh)

In [9]:
output = np.abs(function_result)

In [11]:

fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter(x, y, output, alpha=0.2)

ax.set_xlabel('Real axis')      # 실수부
ax.set_ylabel('Imaginary axis') # 허수부
ax.set_zlabel('Absolute value') # 함수값의 절대값
ax.set_title('One Iteration: $f(z) = z^2 - 1$')

Text(0.5, 0.92, 'One Iteration: $f(z) = z^2 - 1$')

![alt text](Figure_1.png)

In [12]:

output = np.abs(f(f(mesh))) # 함수 두 번 반복

ax = plt.axes(projection='3d')

ax.scatter(x, y, output, alpha=0.2)

ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')
ax.set_zlabel('Absolute value')
ax.set_title('Two Iterations: $ f(z) = z^2 - 1$')

Text(0.5, 0.92, 'Two Iterations: $ f(z) = z^2 - 1$')

![alt text](Figure_2.png)


유사한 복소수 집합에 대해 함수 반복 적용하기

In [13]:
selected_values = np.array([0.4 + 0.4j, 0.41 + 0.4j, 0.4 + 0.41j])
num_iter = 9

outputs = np.zeros(
    (num_iter+1, selected_values.shape[0]),  # 10X3 Zero 배열 생성
    dtype=complex  # 데이터 타입: 복소수
)
outputs[0] = selected_values   # 함수 초기 조건

for i in range(num_iter):
    outputs[i+1] = f(outputs[i])  # 10번 함수 반복하는 동안 각 결과를 저장

fig, axes = plt.subplots(1, selected_values.shape[0], figsize=(16, 6))
axes[1].set_xlabel('Real axis')
axes[0].set_ylabel('Imaginary axis')

for ax, data in zip(axes, outputs.T):
    cycle = ax.scatter(
        data.real,  # 실수부
        data.imag,  # 허수부
        c=range(data.shape[0]), 
        alpha=0.6
    )
    ax.set_title(f'Mapping of iterations on {data[0]}')

fig.colorbar(cycle, ax=axes, location='bottom', label='Iteration');

![alt text](Figure_3.png)


발산 전까지 함수 반복 적용 및 시각화

In [14]:
def divergence_rate(mesh, num_iter=10, radius=2):

    z = mesh.copy()
    # 값 별 반복 횟수를 저장
    diverge_len = np.zeros(mesh.shape)

    # 발산되지 않은 값들에 대해 함수를 반복 적용
    for i in range(num_iter):
        
        # [+] 1. Boolean 배열  마스크 생성
        conv_mask = np.abs(z) < radius

        # [+] 2. 해당 값들에 대한 diverge_len 값 증가
        diverge_len[conv_mask] += 1

        # [+] 3. 해당 값들에 대해 함수 적용
        z[conv_mask] = f(z[conv_mask])

    return diverge_len

In [15]:

x, y = np.meshgrid(np.linspace(-2, 2, 400), np.linspace(-2, 2, 400))
mesh = x + (1j * y)

output = divergence_rate(mesh)

fig = plt.figure(figsize=(5, 5))
ax = plt.axes()

ax.set_title('$f(z) = z^2-1$')
ax.set_xlabel('Real axis')
ax.set_ylabel('Imaginary axis')

im = ax.imshow(output, extent=[-2, 2, -2, 2])
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.1)
plt.colorbar(im, cax=cax, label='Number of iterations');

![alt text](Figure_4.png)